In [1]:
import torch
from model import load_resnet_model
import os
from train_regression_weighted_loss_loaded import train_model
import re
from weighted_MSE_loss import DenseWeight
from dataloader import create_dataloader
from tqdm import tqdm

2024-01-26 09:13:47.766682: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-26 09:13:47.766730: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-26 09:13:47.767514: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-26 09:13:47.773844: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-26 09:13:48.503038: W tensorflow/compiler/tf2

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = load_resnet_model('resnet50', num_classes=1)

/opt/conda/envs/pytorch/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
def get_file_paths(directory):
    """
    Get all file paths in the specified directory with the specified file extension.

    Args:
    - directory (str): The directory to search for files.
    - file_extension (str): The file extension to filter by.

    Returns:
    - List[str]: A list of file paths.
    """
    file_paths = []
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        file_paths.append(file_path)
    return file_paths

# Use a regular expression to extract the batch number from the filename
def extract_batch_number(file_path):
    match = re.search(r"data_batch_(\d+)", file_path)
    if match:
        return int(match.group(1))
    else:
        return -1  # If for some reason a file doesn't match the pattern


In [4]:
train_files = sorted(get_file_paths('./data/records/train/'), key= extract_batch_number)
val_files = sorted(get_file_paths('./data/records/val/'), key= extract_batch_number)

train_loaders = [create_dataloader(file, True, 64) for file in train_files]
val_loaders = [create_dataloader(file, True, 64) for file in val_files]

In [ ]:
training_labels = []
for train_loader in train_loaders:
    for _, labels in tqdm(train_loader):
        training_labels.extend(labels.detach().cpu().numpy())

dense_weight_model = DenseWeight(0.2)
dense_weight_model.fit(training_labels)

In [6]:
train_model(model, train_loaders, val_loaders, device, dense_weight_model,epochs=15)

100%|██████████| 3/3 [00:00<00:00,  8.02it/s]


Epoch 1/15, Train Loss: 0.1247, Train R2: -0.4958, Val Loss: 0.1439, Val R2: -0.3785
Model saved as checkpoint_epoch_1.pth


100%|██████████| 3/3 [00:00<00:00,  9.97it/s]


Epoch 2/15, Train Loss: 0.0655, Train R2: 0.3688, Val Loss: 0.0725, Val R2: 0.3238
Model saved as checkpoint_epoch_2.pth


100%|██████████| 3/3 [00:00<00:00,  9.97it/s]


...

Epoch 10/15, Train Loss: 0.0391, Train R2: 0.6223, Val Loss: 0.0579, Val R2: 0.4464


100%|██████████| 3/3 [00:00<00:00,  9.85it/s]


Epoch 11/15, Train Loss: 0.0439, Train R2: 0.5755, Val Loss: 0.0726, Val R2: 0.3195


100%|██████████| 8/8 [00:02<00:00,  3.49it/s]


KeyboardInterrupt: 